In [5]:
from pyimzml.ImzMLParser import ImzMLParser
from pathlib import Path
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
from tqdm import tqdm

# --- File path ---
data_dir = Path("../data")
imzml_path = data_dir / "/home/parsa/PycharmProjects/HT-SpaceM/maldi_data/imzML_data/hfd_4.imzML"  # <-- Change this to your file

# --- Load imzML ---
parser = ImzMLParser(imzml_path)

# --- Build coordinate metadata ---
coords = np.array(parser.coordinates)
obs = pd.DataFrame(coords, columns=["x", "y", "z"])
obs.index = [f"px_{i}" for i in range(len(coords))]

# --- Bin m/z values ---
min_mz = 100
max_mz = 1000
bin_size = 1.0
bins = np.arange(min_mz, max_mz, bin_size)
var = pd.DataFrame({"m/z_bin": bins})
var.index = [f"mz_{i}" for i in range(len(bins))]

X = np.zeros((len(parser.coordinates), len(bins)))

for i, coord in tqdm(enumerate(parser.coordinates), total=len(parser.coordinates)):
    mzs, intensities = parser.getspectrum(i)
    mzs = np.array(mzs)
    intensities = np.array(intensities)
    bin_indices = np.floor((mzs - min_mz) / bin_size).astype(int)
    mask = (bin_indices >= 0) & (bin_indices < len(bins))
    for idx, val in zip(bin_indices[mask], intensities[mask]):
        X[i, idx] += val

# --- Create AnnData ---
adata = ad.AnnData(X=X, obs=obs, var=var)
adata.layers["counts"] = adata.X.copy()

# --- Filter cells/ions ---
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=50)

# --- Normalize & log-transform ---
sc.pp.normalize_total(adata, target_sum=10000, exclude_highly_expressed=True, max_fraction=0.05)
adata.layers["norm_counts"] = adata.X.copy()
sc.pp.log1p(adata)
adata.layers["log1p"] = adata.X.copy()
adata.raw = adata

# --- Save for scanpy ---
adata.write(data_dir / "hfd_4_preprocessed.h5ad")


/home/parsa/anaconda3/envs/HTSpaceM/lib/python3.13/site-packages/pyimzml/ontology/ontology.py:92: UserWarning: Accession IMS:1000046 found with incorrect name "pixel size x". Updating name to "pixel size (x)".
  warn(
100%|██████████| 8958/8958 [00:02<00:00, 3069.23it/s]
